In [1]:
import numpy as np, matplotlib.pyplot as plt, matplotlib as mpl
import FITS_tools
from astropy.io import fits
import astropy.wcs as pywcs
%matplotlib inline

In [2]:
def load_from_fits(f):
    hdu_list = fits.open(f)
    im = hdu_list['sci'].data
    h = hdu_list['sci'].header
    wcs = pywcs.WCS(h, fobj=hdu_list)
    wcs.naxis2 = h['naxis2']
    wcs.naxis1 = h['naxis1']
    return im, h, wcs

In [3]:
def update_header(h1, h2):
    h3 = h1.copy()
    for k in ['NAXIS1', 'NAXIS2', 'CRPIX1', 'CRPIX2', 'CRVAL1', 'CRVAL2', 
             'CD1_1', 'CD1_2', 'CD2_1', 'CD2_2', 'ORIENTAT', 'RA_APER', 'DEC_APER']:
        h3[k] = h2[k]
    return h2

In [4]:
def shift_fits(file_1, file_2, output_file=None):
    im1, h1, wcs1 = load_from_fits(file_1)
    im2, h2, wcs2 = load_from_fits(file_2)
    print('beginning transformation')
    new_im = FITS_tools.hcongrid.hastrom(im1, wcs1, wcs2, preserve_bad_pixels=True)
    print('transformation complete')
    new_h = update_header(h1, h2)
    hdu = fits.ImageHDU(data=new_im, header=new_h)
    if output_file is not None:
        print('saving result')
        hdu.writeto(output_file)
    return hdu

In [5]:
%%time
shift_fits('../data/m31/raw/hlsp_phat_hst_acs-wfc_12058-m31-b01_f475w_v1_drz.fits',
           '../data/m31/raw/hlsp_phat_hst_acs-wfc_12058-m31-b01_f814w_v1_drz.fits',
        output_file='../data/m31/raw/hlsp_phat_hst_acs-wfc_12058-m31-b01_f475w_v1_drz_ALIGNED.fits')

CPU times: user 3min 46s, sys: 2min 6s, total: 5min 53s
Wall time: 7min 31s


In [ ]:
%%time
shift_fits('../data/m31/raw/hlsp_phat_hst_acs-wfc_12105-m31-b06_f475w_v1_drz.fits',
           '../data/m31/raw/hlsp_phat_hst_acs-wfc_12105-m31-b06_f814w_v1_drz.fits',
        output_file='../data/m31/raw/hlsp_phat_hst_acs-wfc_12105-m31-b06_f475w_v1_drz_ALIGNED.fits')